# Processing IMAGE
Ipynb yang ini untuk : 
1. Download image dari image_url -> save ke os (image didownload sesuai index)
2. cek gambar yang gagal -> sesuaikan sama data clean_text
3. cek mismatch drop index clean_text kalo ada gambar yang ga bisa terload

### CEK JUMLAH ROW IMAGE 
dari dataset cleaned_text_asli 

In [14]:
import pandas as pd

# Tentukan path ke file dataset Anda
file_path = r'E:\$7th\TA\Multimodal_Process_Exploration\DATA\Cleaned_DropNull.csv'

# Muat dataset ke dalam DataFrame pandas
try:
    df = pd.read_csv(file_path)
    print("Dataset berhasil dimuat.")
    print("-" * 30)
    
    # Cek ringkasan dataset
    total_rows = len(df)
    image_url_count = df['image_url'].notna().sum()
    unique_image_url_count = df['image_url'].nunique()

    print(f"Total baris dalam dataset: {total_rows}")
    print(f"Jumlah baris dengan image_url: {image_url_count}")
    print(f"Jumlah image_url unik: {unique_image_url_count}")
    
except FileNotFoundError:
    print(f"ERROR: File tidak ditemukan di path '{file_path}'. Mohon periksa kembali path file Anda.")

Dataset berhasil dimuat.
------------------------------
Total baris dalam dataset: 16526
Jumlah baris dengan image_url: 16526
Jumlah image_url unik: 16208


### DOWNLOAD GAMBAR
dari kolom image_url

In [1]:
import os
import requests
import pandas as pd
from tqdm import tqdm

In [ ]:
# --- Setup Folder Penyimpanan ---
# Tentukan path folder untuk menyimpan gambar
save_dir = r'E:\$7th\TA\Multimodal_Process_Exploration\DATA\image_url'

# Buat folder jika belum ada, exist_ok=True berarti tidak akan error jika folder sudah ada
os.makedirs(save_dir, exist_ok=True)
print(f"Gambar akan disimpan di folder: {save_dir}")


# --- Fungsi untuk Mengunduh Satu Gambar ---
def download_image(row):
    # Ambil URL dari baris saat ini
    url = row['image_url']
    
    # Periksa apakah URL valid
    if not isinstance(url, str) or not url.startswith('http'):
        return 'invalid_url'
        
    try:
        # Buat nama file unik berdasarkan indeks baris DataFrame
        # .name akan mengambil nilai index saat menggunakan .apply()
        filename = f"{row.name}.jpg"
        save_path = os.path.join(save_dir, filename)
        
        # Lakukan request untuk mengunduh gambar, timeout 15 detik
        response = requests.get(url, stream=True, timeout=15)
        
        # Jika request berhasil (kode status 200)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
            return save_path # Kembalikan path file lokal jika berhasil
        else:
            return f'http_error_{response.status_code}'
            
    except requests.exceptions.RequestException as e:
        # Tangani error koneksi, timeout, dll.
        return 'download_failed'

# --- Terapkan Fungsi Download ke Setiap Baris ---
# Inisialisasi tqdm untuk pandas
tqdm.pandas(desc="Downloading Images")

# Terapkan fungsi download_image ke setiap baris di DataFrame (axis=1)
# Hasilnya (path file lokal atau status error) akan disimpan di kolom baru 'local_path'
df['local_path'] = df.progress_apply(download_image, axis=1)


# --- Tampilkan Ringkasan Hasil Download ---
print("\nProses pengunduhan selesai.")
print("-" * 30)

# Hitung jumlah gambar yang berhasil diunduh
# (yaitu yang path-nya diawali dengan save_dir dan bukan status error)
success_count = df['local_path'].str.startswith(save_dir, na=False).sum()
total_images = df['image_url'].notna().sum()

print(f"Total URL gambar yang valid: {total_images}")
print(f"Berhasil diunduh: {success_count}")
print(f"Gagal diunduh: {total_images - success_count}")

print("\n5 baris pertama data dengan path gambar lokal:")
print(df.head())

Gambar akan disimpan di folder: E:\$7th\TA\Multimodal_Process_Exploration\DATA\image_url


### CHECK TOTAL IMAGE SETELAH DOWNLOAD
 (ada beberapa gambar yang missing / ga bisa ke download)

In [21]:
import os 

# --- 1. Ambil semua index gambar yang ada ---
img_dir = r'E:\$7th\TA\Multimodal_Process_Exploration\DATA\image_url'
valid_ext = {".jpg"}


image_files = [f for f in os.listdir(img_dir) if os.path.splitext(f)[1].lower() in valid_ext]
image_indices = sorted([int(os.path.splitext(f)[0]) for f in image_files if os.path.splitext(f)[0].isdigit()])

print("Total file gambar:", len(image_files))
print("Jumlah gambar ditemukan dengan index berurutan:", len(image_indices))
print("Index range:", min(image_indices), "sampai", max(image_indices))



Total file gambar: 16305
Jumlah gambar ditemukan dengan index berurutan: 16305
Index range: 0 sampai 16525


### DROP ROW CLEAN_TEXT based on IMAGE
karena ada yang missing text yang sudah di clean juga di drop based on index yang missing di image. --

tujuannya agar index image dan text masih selaras

In [22]:

# --- 2. Load dataset asli ---
csv_asli = r'E:\$7th\TA\Multimodal_Process_Exploration\DATA\Cleaned_Text.csv'
df = pd.read_csv(csv_asli)

print("Jumlah row awal:", len(df))

# --- 3. Drop row yang tidak ada gambarnya ---
df_cleaned = df[df.index.isin(image_indices)].copy()

print("Jumlah row setelah drop:", len(df_cleaned))
print("Total row yang dibuang:", len(df) - len(df_cleaned))

# --- 4. Simpan hasil (index asli tetap ikut disimpan) ---
csv_output = r'E:\$7th\TA\Multimodal_Process_Exploration\DATA\Cleaned_Text.csv'
df_cleaned.to_csv(csv_output, index=True)   # index disimpan di CSV

print("File baru disimpan di:", csv_output)

df_cleaned.columns

Jumlah row awal: 16526
Jumlah row setelah drop: 16305
Total row yang dibuang: 221
File baru disimpan di: E:\$7th\TA\Multimodal_Process_Exploration\DATA\Cleaned_Text.csv


Index(['full_text', 'clean_text', 'tweet_url', 'image_url', 'user_id_str',
       'conversation_id_str', 'created_at', 'MM/DD/YYYY', 'age_account',
       'username', 'official_account', 'verified_account', 'quote_count',
       'reply_count', 'retweet_count', 'favorite_count', 'followers_count',
       'image_corelation', 'Label'],
      dtype='object')

### CEK MISMATCH GAMBAR dengan CLEANED TEXT

In [23]:
import os
import pandas as pd

# --- 1. Ambil semua index gambar ---
img_dir = r'E:\$7th\TA\Multimodal_Process_Exploration\DATA\image_url'
valid_ext = {".jpg"}

image_files = [f for f in os.listdir(img_dir) if os.path.splitext(f)[1].lower() in valid_ext]
image_indices = sorted([int(os.path.splitext(f)[0]) for f in image_files if os.path.splitext(f)[0].isdigit()])


# --- 2. Load dataset Cleaned_Text.csv ---
csv_path = r'E:\$7th\TA\Multimodal_Process_Exploration\DATA\Cleaned_Text.csv'
df = pd.read_csv(csv_path, index_col=0)   # pakai index asli
mask_empty = df["clean_text"].isna() | (df["clean_text"].astype(str).str.strip() == "")
text_indices = df.index[~mask_empty].tolist()

print(f"Jumlah row text: {len(text_indices)}")
print(f"Jumlah gambar ditemukan: {len(image_indices)}")
#print(f"Range index gambar: {min(image_indices)} - {max(image_indices)}")
#print(f"Range index text: {min(text_indices)} - {max(text_indices)}")

# --- 3. Bandingkan index ---
missing_in_text = sorted(set(image_indices) - set(text_indices))
missing_in_image = sorted(set(text_indices) - set(image_indices))

print("\n=== HASIL PENGECEKAN ===")
if not missing_in_text and not missing_in_image:
    print("✅ Semua index gambar dan text cocok 1:1.")
else:
    print("⚠ Ada mismatch antara text dan image:")
    if missing_in_text:
        print(f"  - Index gambar yang tidak ada di text: {missing_in_text}")
    if missing_in_image:
        print(f"  - Index text yang tidak ada gambar: {missing_in_image}")


Jumlah row text: 16305
Jumlah gambar ditemukan: 16305

=== HASIL PENGECEKAN ===
✅ Semua index gambar dan text cocok 1:1.
